In [ ]:
import pandas as pd
import re
from pykospacing import Spacing
from hanspell import spell_checker
import csv

spacing = Spacing()

# 맞춤법 교정 함수
def safe_spell_check(text):
    try:
        if isinstance(text, str) and 1 < len(text) <= 500:
            result = spell_checker.check(text)
            if hasattr(result, 'checked'):
                return result.checked
    except:
        pass
    return text

def preprocess_text(text):
    if not isinstance(text, str):
        return text

    # 1. 날짜 마스킹
    text = re.sub(r"\d{1,2}\.\d{1,2}\s*[\-~]\s*\d{1,2}\.\d{1,2}(일)?", "DATE", text)
    text = re.sub(r"\d{4}년\s?\d{1,2}월\s?\d{1,2}일", "DATE", text)
    text = re.sub(r"\d{1,2}월\s?\d{1,2}일?", "DATE", text)
    text = re.sub(r"\d{1,4}년\s?\d{1,2}월", "DATE", text)
    text = re.sub(r"\d{1,2}/\d{1,2}\(.\)", "DATE", text)
    text = re.sub(r"\d{1,2}/\d{1,2}\(\w+\)", "DATE", text)
    text = re.sub(r"\d{1,2}일", "DATE", text)
    text = re.sub(r"\d{1,4}년", "DATE", text)
    text = re.sub(r"\d{1,2}월", "DATE", text)

    # 2. 시간 마스킹
    text = re.sub(r"\d{2}:\d{2}", "TIME", text)
    text = re.sub(r"\d{1,2}시\s*[\-~]\s*\d{1,2}시", "TIME", text)
    text = re.sub(r"\d{1,2}시간", "TIME", text)
    text = re.sub(r"\d{1,2}시", "TIME", text)

    # 3. % 앞 숫자 → NUM
    text = re.sub(r"\b\d+(\.\d+)?(?=%)", "NUM", text)

    # 4. 문장 끝 3~4자리 숫자 제거 (줄 끝에서 공백 + 숫자)
    text = re.sub(r"\s\d{3,4}(?=\s|$)", "", text)

    # 5. 고객센터 번호 → NUM (예: 1588-1234, 1544-5678)
    text = re.sub(r"\b(1[5-6]\d{2})[- ]?\d{4}\b", "NUM", text)

    # 6. 특수문자 제거 (한글, 영문, 숫자, 공백 외)
    text = re.sub(r"[^\w\s가-힣]", " ", text)

    # 7. 공백 압축
    text = re.sub(r"\s+", " ", text).strip()

    # 8. 띄어쓰기 보정
    text = spacing(text)

    # 9. 맞춤법 검사
    text = safe_spell_check(text)

    return text

df = pd.read_csv("normal_messages2.csv")

df['content'] = df['content'].apply(preprocess_text)

# 중복 제거
df = df.drop_duplicates(subset=["clean_df"]).copy()

# 저장
df[["SMS","clean_df"]].to_csv("normal_messages_clean_test.csv", index=False, encoding="utf-8-sig", quoting=csv.QUOTE_ALL)